<a href="https://colab.research.google.com/github/muhammadadeel22/Large_language_model_Ass3/blob/main/Assignment3_LLM_ARM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Fou

In [53]:
!git config --global user.email "m.adeel951321@gmail.com"
!git config --global user.name "muhammadadeel22"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [6]:
text = "This is a great [MASK]."

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=512)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
import torch

inputs = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt"
)
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, default_data_collator
from datasets import load_dataset

In [10]:
dataset = load_dataset("AmirLayegh/tacred_text_label")

Generating train split:   0%|          | 0/68124 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/22631 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15509 [00:00<?, ? examples/s]

In [11]:
print(dataset['train'].features)
dataset

{'text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None)}


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 68124
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 22631
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 15509
    })
})

In [12]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: According to an FBI affidavit , Headley admitted working with [subject_start] Lashkar-e-Taiba [subject_end] , knowing that it had been designated by the U.S. government as a foreign terrorist organization , and with Ilyas Kashmiri , a leader of another [object_start] Pakistan-based [object_end] terrorist organization , Harakat ul Jihad Islami .  [subject_start] Lashkar-e-Taiba [subject_end] is Pakistani terrorist organization and [object_start] Pakistan-based [subject_end] is sovereign state in South Asia.'
'>>> Label: na'

'>>> Review: The series leading to the May 1 Derby will begin March 27 on [subject_start] USA Network [subject_end] with the Louisiana Derby from New Orleans , and will also include the Lane 's End Stakes from [object_start] Turfway Park [object_end] in Florence , Ky. .  [subject_start] USA Network [subject_end] is American cable television channel and [object_start] Turfway Park [subject_end] is American horse racing track.'
'>>> Label: na'

'>>> Revi

In [13]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/68124 [00:00<?, ? examples/s]

Map:   0%|          | 0/22631 [00:00<?, ? examples/s]

Map:   0%|          | 0/15509 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 68124
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 22631
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 15509
    })
})

In [14]:
chunk_size = 128

In [15]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 105'
'>>> Review 1 length: 105'
'>>> Review 2 length: 87'


In [16]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 297'


In [17]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 41'


In [18]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/68124 [00:00<?, ? examples/s]

Map:   0%|          | 0/22631 [00:00<?, ? examples/s]

Map:   0%|          | 0/15509 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 58021
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 18765
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 12668
    })
})

In [20]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'ago [ object _ end ] by veteran politician tom thabane, managed to get 17 constituency seats to become a major opposition in the tiny country, with one going to the small alliance of congress parties - lrb - acp - rrb -. [ subject _ start ] abc [ subject _ end ] is political party and [ object _ start ] four months ago [ subject _ end ] is year. [SEP] [CLS] founded in [ object _ start ] 1901 [ object _ end ], the [ subject _ start ] jnf [ subject _ end ] is known around the jewish world for its little blue collection boxes, where jews contributed money'

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [ [MASK] _ start ] zagat [ [MASK] _ end [MASK] survey, the guide empire that started as a hobby for tim and nina [MASK]gat in [ object _ start ] 1979 [ object _ end ] as a two - page typed list of new york restaurants compiled from reviews [MASK] friends, has been put up for [MASK], according to people briefed on the decision. [ subject _ start ] zagat [ [MASK] _ end ] is rating company and [ object [MASK] start ] … [MASK] subject [MASK] end ] is year. [SEP] [CLS] the [ subject _ start [MASK] abc [MASK] subject _ end ], formed only [ object _ start [MASK] four [MASK]'

'>>> ago [ object _ [MASK] ] by veteran politician tom tha [MASK], managed to get 17 constituency [MASK] to become a major opposition in the tiny country, with one going to [MASK] small alliance of congress parties [MASK] lrb - acp - rrb - [MASK] [ subject _ start ] abc [MASK] [MASK] [MASK] end ] is political party and [ [MASK] _ [MASK] ] four months ago [ subject _ end ] is year. [SEP] [CLS] founded in [ obj

In [23]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [24]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [ subject [MASK] start ] zagat [ subject [MASK] end ] survey, the guide empire [MASK] started as a [MASK] for [MASK] and [MASK] zagat in [ [MASK] [MASK] start ] 1979 [ object _ [MASK] ] [MASK] a two - page typed [MASK] of new york restaurants compiled from reviews from friends, has [MASK] put [MASK] for [MASK] [MASK] according to people briefed on the decision [MASK] [ subject _ start ] [MASK] [MASK] [ [MASK] [MASK] end ] is rating [MASK] and [ object [MASK] start ] 1979 [MASK] [MASK] _ end [MASK] is [MASK]. [SEP] [CLS] the [ subject _ [MASK] [MASK] [MASK] [ subject [MASK] end [MASK], [MASK] [MASK] [ object _ [MASK] [MASK] four months'

'>>> ago [ object _ end ] by veteran [MASK] tom [MASK] [MASK] [MASK] managed [MASK] get [MASK] constituency [MASK] to become a major opposition in the tiny country, [MASK] one going to the small alliance of [MASK] [MASK] - lrb - acp [MASK] rrb -. [ subject _ start ] abc [MASK] subject _ end ] is political party and [ object _ start [MASK] fo

In [25]:
train_size = 5_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 500
    })
})

In [26]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-tacred_text_label",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=False,
    logging_steps=logging_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [28]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 11.38


In [29]:
trainer.push_to_hub()

events.out.tfevents.1722892864.f879d15b38bb.284.0:   0%|          | 0.00/297 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/muhammadadeel/distilbert-base-uncased-tacred_text_label/commit/8c3ad5b70b7e6184ef9aec0eb5d33d16f279c92a', commit_message='End of training', commit_description='', oid='8c3ad5b70b7e6184ef9aec0eb5d33d16f279c92a', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [31]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [32]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [33]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [34]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [35]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [36]:
from huggingface_hub import get_full_repo_name

model_name = "distilbert-base-uncased-tacred_text_label"
repo_name = get_full_repo_name(model_name)
repo_name

'muhammadadeel/distilbert-base-uncased-tacred_text_label'

In [38]:
pip install huggingface_hub


In [47]:
from huggingface_hub import HfApi, snapshot_download

api = HfApi()

# Define the model name and repository name
model_name = "distilbert-base-uncased-tacred_text_label"
repo_name = "muhammadadeel/distilbert-base-uncased-tacred_text_label"

# Download the repository contents
repo_path = snapshot_download(repo_id=repo_name, revision="main", cache_dir=model_name)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [49]:
# Create a repository object to manage the repo
try:
    repo_url = api.create_repo(repo_name, exist_ok=True)
except Exception as e:
    print(f"Repository creation failed: {e}")

In [50]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(repo_path, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(repo_path)

        # Push to the Hub using HfApi
        api.upload_folder(
            repo_id=repo_name,
            folder_path=repo_path,
            commit_message=f"Training in progress epoch {epoch}"
        )

  0%|          | 0/237 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 3.5365220690785963


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

>>> Epoch 1: Perplexity: 3.5365220690785963
>>> Epoch 2: Perplexity: 3.5365220690785963


In [51]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="muhammadadeel/distilbert-base-uncased-tacred_text_label"
)

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [52]:
test = "London is the [MASK] of England."
preds = mask_filler(test)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> london is the capital of england.
>>> london is the city of england.
>>> london is the country of england.
>>> london is the centre of england.
>>> london is the kingdom of england.
